# Run Bot By predict Left & Right motor values (LSTM)

This notebook is to run the robot using left and right motor values predicted from model.  
**Model File**       > HDF5 (.h5)  
**Model Output**     > Left/right motor values  
**Folder Structure** >
<pre style="font-size: 10.0pt; font-family: Arial; line-height: 1; letter-spacing: 0.0pt;" >
<b>ITI110_Project/</b>
|__ <b>data/</b>
|  |__ <b>raw/</b>
|____<b>models/</b>
|    |__<b>{Your Model Name}.h5</b>  
|__ <b>run_bot_hdf5_pred_motors.ipynb</b>
</pre>

# Import Libraries

In [1]:
import traitlets
from IPython.display import display
import ipywidgets
from jetbot import Camera, bgr8_to_jpeg, Robot
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
import time, PIL, io, os, pathlib, cv2
import line_detector

In [2]:
print(tf.__version__)
tf.enable_eager_execution()

1.13.1


In [3]:
## Get latest models

In [4]:
!git -C "./github/models/.git" --work-tree=. pull
!ls ./github/ -R


Already up to date.
./github/:
models

./github/models:
1.13.1

./github/models/1.13.1:
CS_LSTM_5_Aug_Flip_lessdata_detectlane_model_best.h5
CS_LSTM_5_Rev_Aug_Flip_lessdata_detectlane_model_best.h5
CS_LSTM_5_Rev_Aug_Flip_lessdata_model_best.h5
CS_LSTM_5_Rev_model_best.h5
CS_LSTM_model_best_2.h5
CS_LSTM_model_best.h5
CS_LSTM_model_best_linedetect.h5
Johann_Pilotnet2302020_model_best.h5


# Load Model

Load Keras HDF5 file (.h5) into memory for inference

In [5]:
#MODEL_DIR = os.path.join(os.getcwd(),'model')
MODEL_DIR = os.path.join(os.getcwd(),'github/models/1.13.1')
#MODEL_PATH = os.path.join(MODEL_DIR, 'Johann_Pilotnet2302020_model_best.h5') ## Change here if you have different name
#MODEL_PATH = os.path.join(MODEL_DIR, 'CS_LSTM_model_best_linedetect.h5')
MODEL_PATH = os.path.join(MODEL_DIR, 'CS_LSTM_5_Aug_Flip_lessdata_detectlane_model_best.h5')
#MODEL_PATH = os.path.join(MODEL_DIR, 'CS_LSTM_5_Rev_model_best.h5')

model = models.load_model(MODEL_PATH)
print(f'Loaded Model: {MODEL_PATH}')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Loaded Model: /home/jetbot/ITI110/jetbot_reference_code/github/models/1.13.1/CS_LSTM_5_Aug_Flip_lessdata_detectlane_model_best.h5


## Preprocess and Model Parameters
Construct preprocess parameters based on input shape.

# Define Preprocess()

The function is use to pre-process input image into desired format required by model. This function must same with the one using in training model.

In [6]:
## This function have to deploy to jetbot
def preprocess(image, h=224, w=224, crop_top=0.0, color=False): 

    """
    The function is to preprocess image before feeding into model. 
    Current feature include resize image based on h and w, vertical cropping, and change color.
    
    :param image: input imag
    :param h: target height of output image in pixel. Default to 224
    :param w: target width of output image in pixel. Default to 224
    :param crop_top: Determine whether to crop the top part of the image. use ratio value. e.g. 0.5 mean crop top half of image. Default to 0 (no Crop)
    :param color: cv2 color conversion. Default to cv2.COLOR_RGB2YUV. Possible values [cv2.COLOR_BGR2GRAY,cv2.COLOR_RGB2YUV,cv2.COLOR_RGB2GRAY,cv2.COLOR_RGB2GRAY, cv2.COLOR_BGR2RGB, cv2.COLOR_RGB2BGR]
    :return normalized and processed image
    
    """

    height, _, _ = image.shape
    image = image[int(height*crop_top):,:,:]  # remove top half of the image, as it is not relevant for lane following
    if color:
      image = cv2.cvtColor(image, color)  # Convert Color Scheme
    #image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (w,h)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing
    

    return image

In [7]:
## This function have to deploy to jetbot
def preprocess_mask(image, h=224, w=224, crop_top=0.0, color = cv2.COLOR_BGR2GRAY): 

    """
    The function is to preprocess image before feeding into model. 
    Current feature include resize image based on h and w, vertical cropping, and change color.
    
    :param image: input imag
    :param h: target height of output image in pixel. Default to 224
    :param w: target width of output image in pixel. Default to 224
    :param crop_top: Determine whether to crop the top part of the image. use ratio value. e.g. 0.5 mean crop top half of image. Default to 0
    :param color: cv2 color conversion. Default to cv2.COLOR_RGB2YUV. Possible values [cv2.COLOR_BGR2GRAY,cv2.COLOR_RGB2YUV,cv2.COLOR_RGB2GRAY,cv2.COLOR_RGB2GRAY, cv2.COLOR_BGR2RGB, cv2.COLOR_RGB2BGR]
    :return normalized and processed image
    
    """

    height, width, _ = image.shape
    image = image[int(height*crop_top):,:,:]  # remove top half of the image, as it is not relevant for lane following
    image = line_detector.mask_lane(image)
    
    #image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (w,h)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing
    image = image.reshape((image.shape[0],image.shape[1], 1))

    return image

Setup Preprocess() parameters

In [8]:
def process_prediction(prediction):
    predicted_index = np.argmax(prediction)
    result = np.zeros(len(prediction))
    result[predicted_index] = 1
    return result

In [9]:
print(model.summary())

input_shape = list(model.layers[0].input_shape)

IMG_H, IMG_W, CHANNEL = input_shape[-3:len(input_shape)]

T_CROP = 0.3

print(IMG_H,IMG_W,CHANNEL,T_CROP)

#check if channels is 1, if 1 use preprocess mask (extract lines)
preprocess_fn = preprocess

if CHANNEL == 1:
    preprocess_fn = preprocess_mask

COLOR_CONV = cv2.COLOR_BGR2YUV ## Change Here


#check is model is sequence model
IS_SEQ_MODEL = False

SEQUENCES = 1 #default 1 for non sequence models

if len(input_shape) == 4:
    IS_SEQ_MODEL = False
elif len(input_shape) == 5:
    IS_SEQ_MODEL = True
    SEQUENCES = input_shape[1]

print('Sequences:: ', SEQUENCES)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 5, 1152)           130148    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               655872    
_________________________________________________________________
dense_12 (Dense)             (None, 100)               12900     
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 50)                5050      
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                510       
__________

# Define Execute()

This function use to continuously capture image, predict value, and run Robot's steering  
**To Complete**

In [10]:
SPEED = 0.4
angle = 0.0
angle_last = 0.0
images = []


In [11]:
def execute(change):    
    global angle, angle_last, images, SEQUENCES
    image = change['new']
    #image = preprocess(image,IMG_H,IMG_W,T_CROP,COLOR_CONV)
    #image = preprocess_mask(image,IMG_H,IMG_W,T_CROP,COLOR_CONV)
    image = preprocess_fn(image,IMG_H,IMG_W,T_CROP,COLOR_CONV)
    h, w, c = image.shape
    #image = image.reshape(1,h,w,c)
    images.append(image)
    
    print(len(images))
    print(image.shape)
    if len(images) == SEQUENCES:
        print(SEQUENCES)
        print(np.array(images).shape)
        if SEQUENCES == 1: #Non-LSTM Model
            left_right = model.predict(np.array(images))[0]
            left_right = process_prediction(left_right)
            print('test',left_right)
        else: #LSTM-Model
            left_right = model.predict(np.array(images).reshape(1,SEQUENCES,h,w,c))[0]
            left_right = process_prediction(left_right)
        #print(left_right)
        #x_slider.value = round(left_right[0,0],3)
        #y_slider.value = round(left_right[0,1],3)

        speed_slider.value = SPEED

        robot.left_motor.value  = speed_slider.value if left_right[2] == 1 else (speed_slider.value * 0.5)
        robot.right_motor.value = speed_slider.value if left_right[0] == 1 else (speed_slider.value * 0.5)
        print('predic_raw::', left_right)
        print("motor:: ", [robot.left_motor.value,robot.right_motor.value])
        images = []

# Start Robot

Initialize camera and robot instances  
Note that sometimes when we restart the kernels, the camera might not be re-initialized. We might need to run the following in the jetbot terminal.
```bash
$ sudo systemctl restart nvargus-daemon
```

In [12]:
camera = Camera.instance(width=224, height=224)
robot = Robot()

Create Live view and config widgets

In [13]:
image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
#image_preprocess_widget = ipywidgets.Image(format='jpeg', width=IMG_W, height=IMG_H)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

Run code below to execute camera value to execute() function

In [14]:
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
#traitlets.dlink((preprocess_fn(camera.value,IMG_H,IMG_W,T_CROP,COLOR_CONV), 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Execute code below for attach camera to trailets. It will start moving the robot by continuous feed image

In [15]:
display(ipywidgets.HBox([image_widget,y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [16]:
execute({'new': camera.value}) # we call the function once to intialize
#time.sleep(1)  # add a small sleep to make sure frames have finished processing

1
(66, 200, 1)


In [17]:
camera.observe(execute, names='value')
#time.sleep(1)  # add a small sleep to make sure frames have finished processing
#robot.stop()

2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [1. 0. 0.]
motor::  [0.2, 0.4]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [0. 1. 0.]
motor::  [0.2, 0.2]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200,

# Stop Robot

In [18]:
camera.unobserve(execute, names='value')

predic_raw:: [1. 0. 0.]
motor::  [0.2, 0.4]
1
(66, 200, 1)
2
(66, 200, 1)
3
(66, 200, 1)
4
(66, 200, 1)
5
(66, 200, 1)
5
(5, 66, 200, 1)
predic_raw:: [1. 0. 0.]
motor::  [0.2, 0.4]


In [19]:
robot.stop()